In [397]:
class EstadoBloques:
    def __init__(self, p=[], m=['a','b'], l=['b','c'], s=[['a','c']]):
        
        self.pinza = p
        self.mesa = m
        self.libre = l
        self.sobre = s

    def __str__(self):
        r = 'Pinza: {}\nEn mesa: {}\nLibres: {}\nSobre: {}\n'.format(self.pinza,
                [mesa for mesa in self.mesa],
                [libre for libre in self.libre],
                [sobre for sobre in self.sobre])
        
        return r
    
    def __eq__(self,other):
        return self.__str__() == other.__str__()
    
    def __hash__(self):
        return hash(self.__str__())
    
    def coger1(self,x):
        
        r = False
        
        if self.pinza == [] and x in self.libre and x not in self.mesa:
            for s in self.sobre:
                if s[1] == x:
                    y,x = s
                    del self.libre[self.libre.index(x)]
                    del self.sobre[self.sobre.index(s)]
                    self.pinza.append(x)
                    self.libre.append(y)
                    r=True
                    
        return r
                    
                    
    def coger2(self,x):
        r = False
        if self.pinza == [] and x in self.libre and x in self.mesa:
            del self.libre[self.libre.index(x)]
            del self.mesa[self.mesa.index(x)]
            self.pinza.append(x)
            r = True
        return r
    
    def desapilar(self,x):
        r = False
        if x in self.pinza:
            del self.pinza[self.pinza.index(x)]
            self.mesa.append(x)
            self.libre.append(x)
            r = True
        return r
    
    def apilar(self,x,y):
        r=False
        if x in self.pinza and y in self.libre:
            del self.pinza[self.pinza.index(x)]
            del self.libre[self.libre.index(y)]
            self.libre.append(x)
            self.sobre.append([y,x])
            r=True
        return r
    

In [400]:
def is_final(n):
    e = n.getEstado()
    return e.pinza == [] and e.mesa == ['c'] and ['c','b'] in e.sobre and ['b','a'] in e.sobre and e.libre == ['a']

In [408]:
class Nodo:
    
    def __init__(self,estado,padre,g,f):
        self.estado = estado
        self.padre = padre
        self.g = g
        self.f = f
        
    # el estado es un estado
    def getEstado(self):
        return self.estado
    
    # el padre es un nodo
    def getPadre(self):
        return self.padre
    
    def getG(self):
        return self.g
    def getF(self):
        return self.f
    
    def camino(self):
        camino = []
        p = self
        
        while p:
            camino.append(p.getEstado())
            p = p.getPadre()
            
        return list(reversed(camino))
    
    def __lt__(self, other):
        return self.f<other.f
    
    def __eq__(self, other):
        return self.getEstado()==other.getEstado()
    
    def __str__(self):
        return self.estado.__str__()# + 'Padre: ' + self.getPadre().estado.__str__()
        

In [393]:
import queue as queue

class Abiertos():
    def __init__(self):
        self.colaPrioridad = queue.PriorityQueue()
    
    def put(self,nodo):
        self.colaPrioridad.put((nodo.getF(),nodo))
    
    def pop(self):
        return self.colaPrioridad.get()
    
    def empty(self):
        return self.colaPrioridad.empty();
    
    def getNodo(self,estado):
        for elem in self.colaPrioridad.queue:
            if elem[1].getEstado()==estado:
                return elem[1]
        return None           
    
    
    def update(self,nodoViejo,nodoNuevo):
        self.colaPrioridad.queue.remove((nodoViejo.getF(),nodoViejo))
        self.colaPrioridad.put((nodoNuevo.getF(),nodoNuevo))
        
    def getNodes(self):
        return list(map(lambda x:x[1],self.colaPrioridad.queue))
    
    def __str__(self):
        return str(self.colaPrioridad.queue)
        

In [404]:
from copy import deepcopy
cajas = ['a','b','c']

def sucesores(nodo,heuristica):
    # estadoViejo = EstadoBloques(estado.pinza, estado.mesa, estado.libre, estado.sobre) 
    
    estado = nodo.getEstado()
    g = nodo.getG()
    estadoModificar = deepcopy(estado)
    
    listaSucc = []
    
    if estado.pinza:
        p = estado.pinza[0]
        for x in estado.libre:
            boo = estadoModificar.apilar(p,x)
            if boo:
                listaSucc.append(Nodo(estadoModificar,nodo,g+1,heuristica()+g))
                estadoModificar = deepcopy(estado)
        estadoModificar.desapilar(p)
        listaSucc.append(Nodo(estadoModificar,nodo,g+1,heuristica()+g))
        estadoModificar = deepcopy(estado)
    else:
        for x in estado.libre:
            if x in estado.mesa:
                estadoModificar.coger2(x)
                listaSucc.append(Nodo(estadoModificar,nodo,g+1,heuristica()+g))
                estadoModificar = deepcopy(estado)
            else:
                estadoModificar.coger1(x)
                listaSucc.append(Nodo(estadoModificar,nodo,g+1,heuristica()+g))
                estadoModificar = deepcopy(estado)
                
    return listaSucc
    

In [388]:
def dfs(nodoInicial):
    visitados, pila = set(), [Inicial]
    while pila:
        actual = pila.pop()
        
        if actual.getEstado()
        
        if actual.getEstado().is_final():
            return actual
        if actual not in visitados:
            visitados.add(actual)
            suc = sucesores(actual,visitados)
            pila.extend(suc)

In [395]:
# Cargar Celda

def AStar(inicial, sucesoresF, metaF, heuristicoF):
        
    abiertos = Abiertos()
    cerrados = {}
    abiertos.put(inicial)
    
    while not abiertos.empty():
        f,nodo = abiertos.pop()
        cerrados[nodo.getEstado()]=nodo
        if metaF(nodo):
            return nodo.camino()
        sucesores=sucesoresF(nodo,heuristicoF)
        for suc in sucesores:
            if abiertos.getNodo(suc.getEstado()) is None and not suc.getEstado() in cerrados:
                abiertos.put(suc)
            if not abiertos.getNodo(suc.getEstado()) is None:
                nodoViejo=abiertos.getNodo(suc.getEstado())
                if suc.getF()<abiertos.getNodo(suc.getEstado()).getF():
                    abiertos.update(nodoViejo,suc)
            if suc.getEstado() in cerrados:
                if suc.getF()<cerrados[suc.getEstado()].getF():
                    del cerrados[suc.getEstado()]
                    abiertos.put(suc)  
    return []


In [396]:
def heuristicaCajas():
    return 1

In [217]:
prueba = EstadoBloques()
print(prueba)

Pinza: []
En mesa: ['c']
Libres: ['a']
Sobre: [['c', 'b'], ['b', 'a']]



In [218]:
prueba.apilar('a','b')

False

In [219]:
prueba.coger1('c')

False

In [220]:
print(prueba.sobre)

[['c', 'b'], ['b', 'a']]


In [221]:
print(prueba)

Pinza: []
En mesa: ['c']
Libres: ['a']
Sobre: [['c', 'b'], ['b', 'a']]



In [222]:
prueba.desapilar('c')

False

In [223]:
print(prueba)

Pinza: []
En mesa: ['c']
Libres: ['a']
Sobre: [['c', 'b'], ['b', 'a']]



In [224]:
prueba.coger2('b')

False

In [225]:
prueba.apilar('b','c')

False

In [226]:
print(prueba)

Pinza: []
En mesa: ['c']
Libres: ['a']
Sobre: [['c', 'b'], ['b', 'a']]



In [227]:
prueba.is_final()

True

In [228]:
prueba.coger2('a')
prueba.apilar('a','b')

False

In [229]:
print(prueba)

Pinza: []
En mesa: ['c']
Libres: ['a']
Sobre: [['c', 'b'], ['b', 'a']]



In [230]:
prueba.is_final()

True

In [231]:
pru2 = EstadoBloques()

In [232]:
print(pru2)

Pinza: []
En mesa: ['c']
Libres: ['a']
Sobre: [['c', 'b'], ['b', 'a']]



In [233]:
nodo = Nodo(pru2,None)

lista = sucesores(nodo)

In [234]:
for j in lista:
    print(j)

Pinza: ['a']
En mesa: ['c']
Libres: ['b']
Sobre: [['c', 'b']]



In [235]:
sucesores(nodo)

In [236]:
prueba is pru2

False

In [237]:
prueba == pru2

True

In [338]:
pru3 = EstadoBloques([], ['a','b'], ['b','c'], [['a','c']])

In [339]:
prueba == pru3

False

In [340]:
print(pru3)

Pinza: []
En mesa: ['a', 'b']
Libres: ['b', 'c']
Sobre: [['a', 'c']]



In [341]:
s = sucesores(Nodo(pru3,None))

for k in s:
    print(k)
    
len(s)

Pinza: ['b']
En mesa: ['a']
Libres: ['c']
Sobre: [['a', 'c']]

Pinza: ['c']
En mesa: ['a', 'b']
Libres: ['b', 'a']
Sobre: []



2

In [325]:
pru3.coger1('c')

True

In [326]:
print(pru3)

Pinza: ['c']
En mesa: ['a', 'b']
Libres: ['b', 'a']
Sobre: []



In [327]:
nodoInicial = Nodo(pru3,None)
nodoFinal = Nodo(prueba,None)

In [328]:
pru4 = EstadoBloques(['b'],['a'],['c'],[['a', 'c']])



In [331]:
s = sucesores(Nodo(pru4,None))

for k in s:
    print(k)
    
len(s)

Pinza: []
En mesa: ['a']
Libres: ['b']
Sobre: [['a', 'c'], ['c', 'b']]

Pinza: []
En mesa: ['a', 'b']
Libres: ['c', 'b']
Sobre: [['a', 'c']]



2

In [336]:
pru5 = EstadoBloques(['c'],['a', 'b'],['b', 'a'],[])


In [337]:
s = sucesores(Nodo(pru5,None))

for k in s:
    print(k)
    
len(s)

Pinza: []
En mesa: ['a', 'b']
Libres: ['a', 'c']
Sobre: [['b', 'c']]

Pinza: []
En mesa: ['a', 'b']
Libres: ['b', 'c']
Sobre: [['a', 'c']]

Pinza: []
En mesa: ['a', 'b', 'c']
Libres: ['b', 'a', 'c']
Sobre: []



3

In [409]:
pruebaP = EstadoBloques([], ['a','b'], ['b','c'], [['a','c']])
c = Nodo(pruebaP,None,0,0)
i = AStar(c,sucesores,is_final,heuristicaCajas)

In [410]:
for c in i:
    print(c)

Pinza: []
En mesa: ['a', 'b']
Libres: ['b', 'c']
Sobre: [['a', 'c']]

Pinza: ['c']
En mesa: ['a', 'b']
Libres: ['b', 'a']
Sobre: []

Pinza: []
En mesa: ['a', 'b', 'c']
Libres: ['b', 'a', 'c']
Sobre: []

Pinza: ['b']
En mesa: ['a', 'c']
Libres: ['a', 'c']
Sobre: []

Pinza: []
En mesa: ['a', 'c']
Libres: ['a', 'b']
Sobre: [['c', 'b']]

Pinza: ['a']
En mesa: ['c']
Libres: ['b']
Sobre: [['c', 'b']]

Pinza: []
En mesa: ['c']
Libres: ['a']
Sobre: [['c', 'b'], ['b', 'a']]

